# Creating and modelling metallic supercells

In this section we will be concerned with modelling supercells of aluminium.
When dealing with periodic problems there is no unique definition of the
lattice: Clearly any duplication of the lattice along an axis is also a valid
repetitive unit to describe exactly the same system.
This is exactly what a **supercell** is: An $n$-fold repetition along one of the
axes of the original lattice.

The following code achieves this for aluminium:

In [1]:
using DFTK
using LinearAlgebra
using ASEconvert

function aluminium_setup(repeat=1; Ecut=7.0, kgrid=[2, 2, 2])
    a = 7.65339
    lattice = a * Matrix(I, 3, 3)
    Al = ElementPsp(:Al; psp=load_psp("hgh/lda/al-q3"))
    atoms     = [Al, Al, Al, Al]
    positions = [[0.0, 0.0, 0.0], [0.0, 0.5, 0.5], [0.5, 0.0, 0.5], [0.5, 0.5, 0.0]]
    unit_cell = periodic_system(lattice, atoms, positions)

    # Make supercell in ASE:
    # We convert our lattice to the conventions used in ASE, make the supercell
    # and then convert back ...
    supercell_ase = convert_ase(unit_cell) * pytuple((repeat, 1, 1))
    supercell     = pyconvert(AbstractSystem, supercell_ase)

    # Unfortunately right now the conversion to ASE drops the pseudopotential information,
    # so we need to reattach it:
    supercell = attach_psp(supercell; Al="hgh/lda/al-q3")

    # Construct an LDA model and discretise
    # Note: We disable symmetries explicitly here. Otherwise the problem sizes
    #       we are able to run on the CI are too simple to observe the numerical
    #       instabilities we want to trigger here.
    model = model_LDA(supercell; temperature=1e-3, symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid)
end;

    CondaPkg Found dependencies: /home/runner/.julia/packages/ASEconvert/CNQ1A/CondaPkg.toml
    CondaPkg Found dependencies: /home/runner/.julia/packages/PythonCall/wXfah/CondaPkg.toml
    CondaPkg Dependencies already up to date


As part of the code we are using a routine inside the ASE,
the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
for creating the supercell and make use of the two-way interoperability of
DFTK and ASE. For more details on this aspect see the documentation
on Input and output formats.

Write an example supercell structure to a file to plot it:

In [2]:
setup = aluminium_setup(5)
convert_ase(periodic_system(setup.model)).write("al_supercell.png")

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123


Python: None

<img src="https://docs.dftk.org/stable/examples/al_supercell.png" width=500 height=500 />

As we will see in this notebook the modelling of a system generally becomes
harder if the system becomes larger.

- This sounds like a trivial statement as *per se* the cost per SCF step increases
  as the system (and thus $N$) gets larger.
- But there is more to it:
  If one is not careful also the *number of SCF iterations* increases
  as the system gets larger.
- The aim of a proper computational treatment of such supercells is therefore
  to ensure that the **number of SCF iterations remains constant** when the
  system size increases.

For achieving the latter DFTK by default employs the `LdosMixing`
preconditioner [^HL2021] during the SCF iterations. This mixing approach is
completely parameter free, but still automatically adapts to the treated
system in order to efficiently prevent charge sloshing. As a result,
modelling aluminium slabs indeed takes roughly the same number of SCF iterations
irrespective of the supercell size:

[^HL2021]:
   M. F. Herbst and A. Levitt.
   *Black-box inhomogeneous preconditioning for self-consistent field iterations in density functional theory.*
   J. Phys. Cond. Matt *33* 085503 (2021). [ArXiv:2009.01665](https://arxiv.org/abs/2009.01665)

In [3]:
self_consistent_field(aluminium_setup(1); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.298424969166                   -0.85    5.0         
  2   -8.300195403090       -2.75       -1.25    1.0   81.8ms
  3   -8.300431734033       -3.63       -1.89    2.1   96.6ms
  4   -8.300460801664       -4.54       -2.77    3.0    107ms
  5   -8.300464228346       -5.47       -3.09    3.0    151ms
  6   -8.300464427232       -6.70       -3.25    7.9    143ms
  7   -8.300464530318       -6.99       -3.39    7.6    176ms
  8   -8.300464589999       -7.22       -3.54    1.6   96.4ms
  9   -8.300464627902       -7.42       -3.73    2.5    125ms
 10   -8.300464634383       -8.19       -3.82    1.0    115ms
 11   -8.300464642365       -8.10       -4.09    2.1    113ms


In [4]:
self_consistent_field(aluminium_setup(2); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -16.63711774038                   -0.71    6.0         
  2   -16.64115359328       -2.39       -1.15    1.0    201ms
  3   -16.67841892079       -1.43       -1.76    2.6    219ms
  4   -16.67915147688       -3.14       -2.22    4.9    313ms
  5   -16.67926066924       -3.96       -2.61    1.9    238ms
  6   -16.67928416158       -4.63       -3.27    1.9    212ms
  7   -16.67928580862       -5.78       -3.54    5.0    307ms
  8   -16.67928621604       -6.39       -4.10    1.9    199ms


In [5]:
self_consistent_field(aluminium_setup(4); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -33.32720691600                   -0.56    7.8         
  2   -33.33449066382       -2.14       -1.00    1.4    704ms
  3   -33.33596517286       -2.83       -1.72    4.6    918ms
  4   -33.33613772423       -3.76       -2.63    6.9    944ms
  5   -33.33681366196       -3.17       -2.26    6.5    1.06s
  6   -33.33684698180       -4.48       -2.33    2.2    688ms
  7   -33.33694324040       -4.02       -3.59    1.8    728ms
  8   -33.33694359098       -6.46       -3.64    5.2    1.13s
  9   -33.33694376776       -6.75       -4.06    1.6    698ms


When switching off explicitly the `LdosMixing`, by selecting `mixing=SimpleMixing()`,
the performance of number of required SCF steps starts to increase as we increase
the size of the modelled problem:

In [6]:
self_consistent_field(aluminium_setup(1); tol=1e-4, mixing=SimpleMixing());

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.298338750699                   -0.85    5.1         
  2   -8.300240145707       -2.72       -1.59    1.1   60.5ms
  3   -8.300438785131       -3.70       -2.66    2.5   77.0ms
  4   -8.300431747717   +   -5.15       -2.49    3.6    109ms
  5   -8.300464339662       -4.49       -3.43    1.0   75.0ms
  6   -8.300464601785       -6.58       -3.85    2.2   81.9ms
  7   -8.300464641563       -7.40       -4.47    1.5   67.0ms


In [7]:
self_consistent_field(aluminium_setup(4); tol=1e-4, mixing=SimpleMixing());

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -33.29408555160                   -0.56    7.0         
  2   -33.32082703822       -1.57       -1.26    2.1    636ms
  3   -25.27413407266   +    0.91       -0.61    6.1    1.13s
  4   -32.58607683450        0.86       -1.09    5.0    1.10s
  5   -33.23354599683       -0.19       -1.30    2.8    831ms
  6   -32.88868797303   +   -0.46       -1.24    3.4    856ms
  7   -33.03572331186       -0.83       -1.32    4.2    895ms
  8   -33.33433507611       -0.52       -2.24    2.8    737ms
  9   -33.33458911332       -3.60       -2.16    4.1    983ms
 10   -33.33556789525       -3.01       -2.38    2.2    751ms
 11   -33.33685789139       -2.89       -2.77    1.9    650ms
 12   -33.33685361168   +   -5.37      

For completion let us note that the more traditional `mixing=KerkerMixing()`
approach would also help in this particular setting to obtain a constant
number of SCF iterations for an increasing system size (try it!). In contrast
to `LdosMixing`, however, `KerkerMixing` is only suitable to model bulk metallic
system (like the case we are considering here). When modelling metallic surfaces
or mixtures of metals and insulators, `KerkerMixing` fails, while `LdosMixing`
still works well. See the Modelling a gallium arsenide surface example
or [^HL2021] for details. Due to the general applicability of `LdosMixing` this
method is the default mixing approach in DFTK.